In [ ]:
#getting required libraries
!pip install pandas transformers sacrebleu rouge-score datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 50.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.7/119.7 kB 16.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 48.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.7/311.7 kB 35.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 90.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 72.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.7 MB/s eta 0:00:00
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=6fe11416cc3cd8f27392d0fc68f2d234dbf9d846a3d933fe1603e2e8d658d346
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score


In [ ]:
#importting libraries
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import T5ForConditionalGeneration, BartForConditionalGeneration, PegasusForConditionalGeneration, T5Tokenizer, BartTokenizer, PegasusTokenizer
from sacrebleu import corpus_bleu
from rouge_score import rouge_scorer
from transformers import pipeline, set_seed

import matplotlib.pyplot as plt

from datasets import load_dataset, load_metric
from transformers import FlaxAutoModelForSeq2SeqLM, AutoTokenizer

import nltk
from nltk.tokenize import sent_tokenize

nltk.download("punkt")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
#Dataset
import pandas as pd

# Replace 'your_dataset.xlsx' with the path to your local XLSX file
xlsx_file_path = '/content/drive/MyDrive/idp_dataset/pubmed_abstracts_clinicaltextrouge.xlsx'

# Read the XLSX file into a pandas DataFrame
data= pd.read_excel(xlsx_file_path)

# Display the dataset description
print("Dataset Description:")
print(data.describe())

# df

Dataset Description:
         Article ID  Abs vs SAbs  Title vs Abs  Title vs SAbs    ROUGE-1  \
count  4.800000e+01    48.000000     48.000000      48.000000  48.000000   
mean   3.773225e+07     0.854504      0.729341       0.790194   0.163082   
std    5.684680e+03     0.060772      0.051085       0.052805   0.043407   
min    3.772345e+07     0.661455      0.552417       0.643820   0.095008   
25%    3.772685e+07     0.836755      0.709365       0.769479   0.136309   
50%    3.773240e+07     0.855002      0.740741       0.798457   0.155514   
75%    3.773761e+07     0.892681      0.758076       0.831664   0.174604   
max    3.774177e+07     0.948974      0.845376       0.872537   0.299242   

         ROUGE-2    ROUGE-L  
count  48.000000  48.000000  
mean    0.182579   0.772542  
std     0.053278   0.103415  
min     0.113111   0.430303  
25%     0.151770   0.735550  
50%     0.169245   0.787441  
75%     0.196112   0.843756  
max     0.335878   0.904192  


In [ ]:
data.rename(columns = {'Article ID.1':'Article Name','Article ID.2':'Abstract'}, inplace = True)

In [ ]:
# Split the dataset into training and test sets
train_data, test_data = train_test_split(data, test_size=0.25, random_state=42)

In [ ]:
train_data

,Article ID,Article Name,Abstract,Summarized Abstract,Abs vs SAbs,Title vs Abs,Title vs SAbs,ROUGE-1,ROUGE-2,ROUGE-L
6,37739223,Management of postoperative outcomes of polyte...,Guided bone regeneration (GBR) is a validated ...,The use of polytetrafluoroethylene (PTFE) memb...,0.900016,0.764065,0.841933,0.159468,0.167224,0.765690
39,37726017,A Systematic Review and Meta-analysis of Two N...,This paper aims to evaluate whether there is a...,This paper aims to evaluate whether there is a...,0.849829,0.697429,0.802348,0.155989,0.170868,0.768966
33,37728085,TRPM8 as a Potential Biomarker and Therapeutic...,INTRODUCTION: Gastric cancer is a well-known m...,Gastric cancer is a well-known malignant tumor...,0.867510,0.761466,0.836983,0.225092,0.245353,0.637755
13,37736904,Methadone Treatment Gap in Tennessee and How M...,The opioid epidemic has been an ongoing public...,The number of overdose deaths involving opioid...,0.852702,0.574879,0.649035,0.150342,0.153318,0.792135
17,37734884,Psychosocial patient perspectives following ma...,INTRODUCTION: Persistent psychosocial problems...,Persistent psychosocial problems in people wit...,0.910653,0.845376,0.872537,0.167683,0.187117,0.768340
45,37723918,A systematic review of data elements of comput...,OBJECTIVE: Medication errors are the third lea...,Medication errors are the third leading cause ...,0.854706,0.709671,0.789851,0.128151,0.168776,0.772251
15,37736067,Incorporating Genetic Testing Into the Care of...,PURPOSE OF REVIEW: Amyotrophic lateral scleros...,Amyotrophic lateral sclerosis (ALS) and fronto...,0.920217,0.744549,0.769769,0.260274,0.327189,0.450000
9,37738791,A systematic scoping review of patient and car...,OBJECTIVE: We conducted a systematic scoping r...,Review of self-report tools used to measure pa...,0.842720,0.702098,0.836826,0.138037,0.160494,0.850202
16,37735245,The COVID-19 pandemic did not negatively impac...,Outpatient care patterns have changed markedly...,"In 2019-20, 594,350 (98.4%) of 603,877 communi...",0.812543,0.708447,0.838884,0.185039,0.198413,0.732323
29,37730384,Young people's opinions of cancer care in Engl...,OBJECTIVES: The BRIGHTLIGHT cohort study was t...,The BRIGHTLIGHT cohort study was the national ...,0.888808,0.723930,0.835237,0.157107,0.195489,0.727564


In [ ]:
#column names
train_data.columns

Index(['Article ID', 'Article Name', 'Abstract', 'Summarized Abstract',
       'Abs vs SAbs', 'Title vs Abs', 'Title vs SAbs', 'ROUGE-1', 'ROUGE-2',
       'ROUGE-L'],
      dtype='object')

In [ ]:
#sample size of abstract vs summarized_abstract
sample =train_data.iloc[1]
print(f"""
Atricle (except of 500 characters, total length : {len(sample['Abstract'])}):
""")
print(sample['Abstract'][:500])
print(f"\n Summary (length: {len(sample['Summarized Abstract'])}):")
print(sample['Summarized Abstract'])


Atricle (except of 500 characters, total length : 2039):

This paper aims to evaluate whether there is a device-dependent effect on the 
reduction of surgical site complications in obese patients (body mass index 
[BMI] ≥ 30 kg/m2) undergoing cesarean section (C-section). PubMed, Embase, 
Cochrane Library, and ClinicalTrials.gov were searched for the period, January 
2011 to September 2021. English language articles describing a randomized 
controlled trial (RCT) that compared either a -80 or -125 mm Hg single-use 
negative pressure wound therapy (sNPW

 Summary (length: 523):
This paper aims to evaluate whether there is a device-dependent effect on the                 reduction of surgical site complications in obese patients. The primary outcome of interest was surgical site infection (SSI) Secondary outcomes assessed included seroma, dehiscence, hematoma, bleeding, reoperation, readmission, and (composite) wound complications. A statistically significant improvement in the composit

In [ ]:
#TEXT SUMMARIZATION PIPELINES

In [ ]:
sample_text = train_data['Abstract'][1]
#we'll collect generated summaries of each model in a dictionary
summaries ={}

In [ ]:
sample_text

"Radiology reporting is narrative, and its content depends on the clinician's \nability to interpret the images accurately. A tertiary hospital, such as \nanonymous institute, focuses on writing reports narratively as part of training \nfor medical personnel. Nevertheless, free-text reports make it inconvenient to \nextract information for clinical audits and data mining. Therefore, we aim to \nconvert unstructured breast radiology reports into structured formats using \nnatural language processing (NLP) algorithm. This study used 327 de-identified \nbreast radiology reports from the anonymous institute. The radiologist \nidentified the significant data elements to be extracted. Our NLP algorithm \nachieved 97% and 94.9% accuracy in training and testing data, respectively. \nHenceforth, the structured information was used to build the predictive model \nfor predicting the value of the BIRADS category. The model based on random \nforest generated the highest accuracy of 92%. Our study n

In [ ]:
# Convert sample_text to a string if it's not already
if not isinstance(sample_text, str):
    sample_text = str(sample_text)

### **PEGASUS**

In [ ]:
!pip install sentencepiece

In [ ]:
!pip install sentencepiece

from transformers import pipeline

pipe = pipeline('summarization', model="google/pegasus-cnn_dailymail")

pipe_out = pipe(sample_text)

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.encoder.embed_positions.weight', 'model.decoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
pipe_out

[{'summary_text': 'This study used 327 de-identified breast radiology reports from the anonymous institute .<n>The structured information was used to build the predictive model for predicting the value of the BIRADS category .'}]

In [ ]:
summaries["pegasus"] = pipe_out[0]["summary_text"].replace(" .<n>", ".\n")

In [ ]:
summaries["pegasus"]

'This study used 327 de-identified breast radiology reports from the anonymous institute.\nThe structured information was used to build the predictive model for predicting the value of the BIRADS category .'

### **BASELINE MODEL**

In [ ]:
def baseline_summary_three_sent(text):
        return "\n". join(sent_tokenize(text)[:3])

In [ ]:
summaries['baseline'] = baseline_summary_three_sent(sample_text)
summaries['baseline']

"Radiology reporting is narrative, and its content depends on the clinician's \nability to interpret the images accurately.\nA tertiary hospital, such as \nanonymous institute, focuses on writing reports narratively as part of training \nfor medical personnel.\nNevertheless, free-text reports make it inconvenient to \nextract information for clinical audits and data mining."

### **GPT2**

In [ ]:
from transformers import pipeline, set_seed

set_seed(42)
pipe = pipeline('text-generation',model='gpt2-medium')

gpt2_query = sample_text +'\nTL;DR:\n'
pipe_out = pipe(gpt2_query, max_length =512 ,clean_up_tokenization_spaces =True)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [ ]:
pipe_out

[{'generated_text': "Radiology reporting is narrative, and its content depends on the clinician's \nability to interpret the images accurately. A tertiary hospital, such as \nanonymous institute, focuses on writing reports narratively as part of training \nfor medical personnel. Nevertheless, free-text reports make it inconvenient to \nextract information for clinical audits and data mining. Therefore, we aim to \nconvert unstructured breast radiology reports into structured formats using \nnatural language processing (NLP) algorithm. This study used 327 de-identified \nbreast radiology reports from the anonymous institute. The radiologist \nidentified the significant data elements to be extracted. Our NLP algorithm \nachieved 97% and 94.9% accuracy in training and testing data, respectively. \nHenceforth, the structured information was used to build the predictive model \nfor predicting the value of the BIRADS category. The model based on random \nforest generated the highest accuracy

In [ ]:
pipe_out[0]['generated_text']

"Radiology reporting is narrative, and its content depends on the clinician's \nability to interpret the images accurately. A tertiary hospital, such as \nanonymous institute, focuses on writing reports narratively as part of training \nfor medical personnel. Nevertheless, free-text reports make it inconvenient to \nextract information for clinical audits and data mining. Therefore, we aim to \nconvert unstructured breast radiology reports into structured formats using \nnatural language processing (NLP) algorithm. This study used 327 de-identified \nbreast radiology reports from the anonymous institute. The radiologist \nidentified the significant data elements to be extracted. Our NLP algorithm \nachieved 97% and 94.9% accuracy in training and testing data, respectively. \nHenceforth, the structured information was used to build the predictive model \nfor predicting the value of the BIRADS category. The model based on random \nforest generated the highest accuracy of 92%. Our study n

In [ ]:
summaries['gpt2'] = '\n'.join(sent_tokenize(pipe_out[0]['generated_text']))

In [ ]:
summaries['gpt2']

"Radiology reporting is narrative, and its content depends on the clinician's \nability to interpret the images accurately.\nA tertiary hospital, such as \nanonymous institute, focuses on writing reports narratively as part of training \nfor medical personnel.\nNevertheless, free-text reports make it inconvenient to \nextract information for clinical audits and data mining.\nTherefore, we aim to \nconvert unstructured breast radiology reports into structured formats using \nnatural language processing (NLP) algorithm.\nThis study used 327 de-identified \nbreast radiology reports from the anonymous institute.\nThe radiologist \nidentified the significant data elements to be extracted.\nOur NLP algorithm \nachieved 97% and 94.9% accuracy in training and testing data, respectively.\nHenceforth, the structured information was used to build the predictive model \nfor predicting the value of the BIRADS category.\nThe model based on random \nforest generated the highest accuracy of 92%.\nOur 

### **T5**

In [ ]:
pipe = pipeline('summarization', model = 't5-small' )

pipe_out = pipe(sample_text)

In [ ]:
pipe_out

[{'summary_text': 'tertiary hospital, such as anonymous institute, focuses on writing reports narratively as part of training for medical personnel . free-text reports make it inconvenient to extract information for clinical audits and data mining .'}]

In [ ]:
summaries['t5'] = 'n'.join(sent_tokenize(pipe_out[0]['summary_text']))

In [ ]:
summaries['t5']

'tertiary hospital, such as anonymous institute, focuses on writing reports narratively as part of training for medical personnel .nfree-text reports make it inconvenient to extract information for clinical audits and data mining .'

### **BART**

In [ ]:
pipe = pipeline("summarization", model="facebook/bart-large-cnn")
pipe_out = pipe(sample_text)

In [ ]:
pipe_out

[{'summary_text': 'Free-text reports make it inconvenient to extract information for clinical audits and data mining. We aim to convert unstructured breast radiology reports into structured formats using natural language processing (NLP) algorithm. Our NLP algorithm achieved 97% and 94.9% accuracy in training and testing data.'}]

In [ ]:
summaries["bart"] = "\n".join(sent_tokenize(pipe_out[0]["summary_text"]))

In [ ]:
print(summaries["bart"])

Free-text reports make it inconvenient to extract information for clinical audits and data mining.
We aim to convert unstructured breast radiology reports into structured formats using natural language processing (NLP) algorithm.
Our NLP algorithm achieved 97% and 94.9% accuracy in training and testing data.


In [ ]:
## Comparing Different Summaries

In [ ]:
print("GROUND TRUTH")

print(train_data.iloc[1]['Summarized Abstract'])


for model_name in summaries:
    print(model_name.upper())
    print(summaries[model_name])

GROUND TRUTH
This paper aims to evaluate whether there is a device-dependent effect on the                 reduction of surgical site complications in obese patients. The primary outcome of interest was surgical site infection (SSI) Secondary outcomes assessed included seroma, dehiscence, hematoma, bleeding, reoperation, readmission, and (composite) wound complications. A statistically significant improvement in the composite SSI and superficial SSI outcomes was observed with the -80 mm Hg device, compared with standard dressings.
PEGASUS
This study used 327 de-identified breast radiology reports from the anonymous institute.
The structured information was used to build the predictive model for predicting the value of the BIRADS category .
BASELINE
Radiology reporting is narrative, and its content depends on the clinician's 
ability to interpret the images accurately.
A tertiary hospital, such as 
anonymous institute, focuses on writing reports narratively as part of training 
for medi

### ***EVALUATION---***

### **SacreBLEU**

In [ ]:
from datasets import load_metric

bleu_metric = load_metric("sacrebleu")

In [ ]:
#t5
import numpy as np
bleu_metric.add(prediction = [summaries["t5"]], reference = [train_data.iloc[1]['Summarized Abstract'] ])

results = bleu_metric.compute(smooth_method = 'floor', smooth_value = 0 )

results['precision'] = [np.round(p , 2) for
                        p in results['precisions'] ]

pd.DataFrame.from_dict(results, orient = 'index', columns = ['Value'] )

,Value
score,0.0
counts,"[8, 0, 0, 0]"
totals,"[39, 38, 37, 36]"
precisions,"[20.512820512820515, 0.0, 0.0, 0.0]"
bp,0.323614
sys_len,39
ref_len,83
precision,"[20.51, 0.0, 0.0, 0.0]"


In [ ]:
#pegasus
import numpy as np
bleu_metric.add(prediction = [summaries["pegasus"]], reference = [train_data.iloc[1]['Summarized Abstract'] ])

results = bleu_metric.compute(smooth_method = 'floor', smooth_value = 0 )

results['precision'] = [np.round(p , 2) for
                        p in results['precisions'] ]

pd.DataFrame.from_dict(results, orient = 'index', columns = ['Value'] )

,Value
score,0.0
counts,"[8, 0, 0, 0]"
totals,"[36, 35, 34, 33]"
precisions,"[22.22222222222222, 0.0, 0.0, 0.0]"
bp,0.271022
sys_len,36
ref_len,83
precision,"[22.22, 0.0, 0.0, 0.0]"


In [ ]:
#bart

import numpy as np
bleu_metric.add(prediction = [summaries["bart"]], reference = [train_data.iloc[1]['Summarized Abstract'] ])

results = bleu_metric.compute(smooth_method = 'floor', smooth_value = 0 )

results['precision'] = [np.round(p , 2) for
                        p in results['precisions'] ]

pd.DataFrame.from_dict(results, orient = 'index', columns = ['Value'] )

,Value
score,0.0
counts,"[9, 0, 0, 0]"
totals,"[56, 55, 54, 53]"
precisions,"[16.071428571428573, 0.0, 0.0, 0.0]"
bp,0.617459
sys_len,56
ref_len,83
precision,"[16.07, 0.0, 0.0, 0.0]"


### **ROUGE**

In [ ]:
rouge_metric = load_metric('rouge')

In [ ]:
rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]

reference = train_data.iloc[1]['Summarized Abstract']

records = []

for model_name in summaries:
    rouge_metric.add(prediction = summaries[model_name], reference = reference )
    score = rouge_metric.compute()
    rouge_dict = dict((rn, score[rn].mid.fmeasure ) for rn in rouge_names )
    print('rouge_dict ', rouge_dict )
    records.append(rouge_dict)

pd.DataFrame.from_records(records, index = summaries.keys() )

rouge_dict  {'rouge1': 0.15841584158415842, 'rouge2': 0.0, 'rougeL': 0.1188118811881188, 'rougeLsum': 0.1188118811881188}
rouge_dict  {'rouge1': 0.14634146341463414, 'rouge2': 0.016528925619834708, 'rougeL': 0.0975609756097561, 'rougeLsum': 0.11382113821138212}
rouge_dict  {'rouge1': 0.14901960784313725, 'rouge2': 0.00790513833992095, 'rougeL': 0.08627450980392158, 'rougeLsum': 0.13333333333333336}
rouge_dict  {'rouge1': 0.07766990291262137, 'rouge2': 0.0, 'rougeL': 0.05825242718446602, 'rougeLsum': 0.05825242718446602}
rouge_dict  {'rouge1': 0.06896551724137931, 'rouge2': 0.0, 'rougeL': 0.06896551724137931, 'rougeLsum': 0.06896551724137931}


,rouge1,rouge2,rougeL,rougeLsum
pegasus,0.158416,0.000000,0.118812,0.118812
baseline,0.146341,0.016529,0.097561,0.113821
gpt2,0.149020,0.007905,0.086275,0.133333
t5,0.077670,0.000000,0.058252,0.058252
bart,0.068966,0.000000,0.068966,0.068966


In [ ]:
#Evaluationg on the TEST set of the Dataset

In [ ]:
def calculate_metric_on_baseline_test_ds(dataset, metric, column_text = 'Abstract', column_summary = 'Summarized Abstract' ):
    """
    This function calculates a specified metric on a baseline test dataset for a Natural Language Processing (NLP) task.
    It assumes the task is a text summarization task, where the goal is to generate a summary (e.g., highlights) from a text (e.g., article).

    Parameters:
    dataset (pandas.DataFrame): The test dataset. It should contain a column for the text and a column for the true summary.
    metric (datasets.Metric): The metric to calculate. This should be a metric object from the Hugging Face datasets library.
    column_text (str, optional): The name of the column in the dataset that contains the text. Defaults to 'article'.
    column_summary (str, optional): The name of the column in the dataset that contains the true summary. Defaults to 'highlights'.

    Returns:
    score (float): The calculated score of the metric on the test dataset.
    """
    summaries = [baseline_summary_three_sent(text) for text in dataset[column_text] ]

    metric.add_batch(predictions = summaries, references = dataset[column_summary] )

    score = metric.compute()
    return score

In [ ]:

test_sampled = test_data.sample(frac=1, random_state=42).head(100)
score = calculate_metric_on_baseline_test_ds(test_sampled, rouge_metric )

rouge_dict = dict((rn, score[rn].mid.fmeasure ) for rn in rouge_names )

pd.DataFrame.from_dict(rouge_dict, orient = 'index' , columns = ['baseline'] ).T

,rouge1,rouge2,rougeL,rougeLsum
baseline,0.493145,0.353682,0.408058,0.448181


### Strategy to calculate the ROUGE Metric on test dataset for the other **Models**

In [ ]:
test_data['Abstract'].iloc[0]

"INTRODUCTION: Aedes mosquitoes are the primary vectors for the spread of viruses \nlike dengue (DENV), zika (ZIKV) and chikungunya (CHIKV), all of which affect \nhumans. Those diseases contribute to global public health issues because of \ntheir great dispersion in rural and urban areas. Mathematical and statistical \nmodels have become helpful in understanding these diseases' epidemiological \ndynamics. However, modelling the complexity of a real phenomenon, such as a \nviral disease, should consider several factors. This scoping review aims to \ndocument, identify and classify the most important factors as well as the \nmodelling strategies for the spread of DENV, ZIKV and CHIKV.\nMETHODS AND ANALYSIS: We will conduct searches in electronic bibliographic \ndatabases such as PubMed, MathSciNet and the Web of Science for full-text \npeer-reviewed articles written in English, French and Spanish. These articles \nshould use mathematical and statistical modelling frameworks to study deng

In [ ]:
test_data.columns

Index(['Article ID', 'Article Name', 'Abstract', 'Summarized Abstract',
       'Abs vs SAbs', 'Title vs Abs', 'Title vs SAbs', 'ROUGE-1', 'ROUGE-2',
       'ROUGE-L'],
      dtype='object')

In [ ]:
from tqdm import tqdm
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"

def generate_batch_sized_chunks(list_of_elements, batch_size):
    """split the dataset into smaller batches that we can process simultaneously
    Yield successive batch-sized chunks from list_of_elements.

    Generator function to yield successive batch-sized chunks from list_of_elements.

    Parameters:
    list_of_elements (list): List of elements to be divided into chunks.
    batch_size (int): The size of each chunk.

    Yields:
    list: Batch-sized chunk from list_of_elements.

    """
    for i in range(0, len(list_of_elements), batch_size):
        yield list_of_elements[i : i + batch_size]

def calculate_metric_on_test_ds(dataset, metric, model, tokenizer,
                               batch_size=16, device=device,
                               column_text="Abstract",
                               column_summary="Summarized Abstract"):
    """
    Function to calculate a specified metric on a test dataset for a Natural Language Processing (NLP) task.
    It assumes the task is a text summarization task, where the goal is to generate a summary from a text.

    Parameters:
    dataset (pandas.DataFrame): The test dataset. It should contain a column for the text and a column for the true summary.
    metric (datasets.Metric): The metric to calculate. This should be a metric object from the Hugging Face datasets library.
    model (transformers.PreTrainedModel): The transformer model to use for text generation.
    tokenizer (transformers.PreTrainedTokenizer): The tokenizer corresponding to the model.
    batch_size (int, optional): The size of the batches to use for processing. Defaults to 16.
    device (str, optional): The device to run the model on. Defaults to the output of torch.cuda.is_available().
    column_text (str, optional): The name of the column in the dataset that contains the text. Defaults to 'article'.
    column_summary (str, optional): The name of the column in the dataset that contains the true summary. Defaults to 'highlights'.

    Returns:
    score (float): The calculated score of the metric on the test dataset.
    """
    article_batches = list(generate_batch_sized_chunks(dataset[column_text], batch_size))
    target_batches = list(generate_batch_sized_chunks(dataset[column_summary], batch_size))

    for article_batch, target_batch in tqdm(
        zip(article_batches, target_batches), total=len(article_batches)):
        inputs = tokenizer([article_batch], max_length=1024, truncation=True, padding="max_length", return_tensors="pt")

        summaries = model.generate(input_ids=inputs["input_ids"].to(device),
                         attention_mask=inputs["attention_mask"].to(device),
                         length_penalty=0.8, num_beams=8, max_length=128)
        ''' parameter for length penalty ensures that the model does not generate sequences that are too long. '''

        # Finally, we decode the generated texts,
        # replace the <n> token, and add the decoded texts with the references to the metric.
        decoded_summaries = [tokenizer.decode(s, skip_special_tokens=True,
                                clean_up_tokenization_spaces=True)
               for s in summaries]

        decoded_summaries = [d.replace("<n>", " ") for d in decoded_summaries]


        metric.add_batch(predictions=decoded_summaries, references=target_batch)

    #  Finally compute and return the ROUGE scores.
    score = metric.compute()
    return score

In [ ]:
test = test_data['Abstract'].iloc(0)

# Convert sample_text to a string if it's not already
if not isinstance(test, str):
   test = str(test)

In [ ]:
# Import the necessary libraries
from rouge_score import rouge_scorer

# Create a ROUGE scorer
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL', 'rougeLsum'], use_stemmer=True)

# Initialize lists to store ROUGE scores for each model on training and test data
train_rouge_scores = {'Model': [], 'ROUGE1': [], 'ROUGE2': [], 'ROUGEL': [], 'ROUGELSUM': []}
test_rouge_scores = {'Model': [], 'ROUGE1': [], 'ROUGE2': [], 'ROUGEL': [], 'ROUGELSUM': []}

# Iterate over the training and test data, and evaluate ROUGE scores for each model
for data, rouge_scores in [(train_data, train_rouge_scores), (test_data, test_rouge_scores)]:
    for model_name, generated_summary in summaries.items():
        for index, row in data.iterrows():
            reference_summary = row['Summarized Abstract']
            scores = scorer.score(reference_summary, generated_summary)
            rouge_scores['Model'].append(model_name)
            rouge_scores['ROUGE1'].append(scores['rouge1'].fmeasure)
            rouge_scores['ROUGE2'].append(scores['rouge2'].fmeasure)
            rouge_scores['ROUGEL'].append(scores['rougeL'].fmeasure)
            rouge_scores['ROUGELSUM'].append(scores['rougeLsum'].fmeasure)

# Create DataFrames to store the ROUGE scores
train_rouge_df = pd.DataFrame(train_rouge_scores)
test_rouge_df = pd.DataFrame(test_rouge_scores)

# Display the ROUGE scores for training and test data
print("ROUGE Scores for Training Data:")
print(train_rouge_df.pivot_table(index='Model', values=['ROUGE1', 'ROUGE2', 'ROUGEL', 'ROUGELSUM'], aggfunc='mean'))

print("\nROUGE Scores for Test Data:")
print(test_rouge_df.pivot_table(index='Model', values=['ROUGE1', 'ROUGE2', 'ROUGEL', 'ROUGELSUM'], aggfunc='mean'))


ROUGE Scores for Training Data:
            ROUGE1    ROUGE2    ROUGEL  ROUGELSUM
Model                                            
bart      0.133884  0.020034  0.094861   0.103854
baseline  0.169808  0.015507  0.109960   0.140683
gpt2      0.197787  0.029765  0.122900   0.156519
pegasus   0.161645  0.010471  0.118247   0.123550
t5        0.104898  0.007337  0.073531   0.073531

ROUGE Scores for Test Data:
            ROUGE1    ROUGE2    ROUGEL  ROUGELSUM
Model                                            
bart      0.133109  0.013735  0.092602   0.099444
baseline  0.161564  0.005377  0.104981   0.138405
gpt2      0.195955  0.017871  0.105710   0.156328
pegasus   0.176550  0.018012  0.116628   0.124409
t5        0.110360  0.003163  0.076519   0.076519


In [ ]:
# Define a function to print the actual and generated summaries for each model
def print_actual_vs_generated_summaries(model_name, actual_summary, generated_summary):
    print(f"{model_name.upper()} - Actual Summary:")
    print(actual_summary)
    print(f"{model_name.upper()} - Generated Summary:")
    print(generated_summary)
    print("\n")

# Iterate through the models and print the actual vs generated summaries without truncation
for model_name in summaries:
    actual_summary = train_data.iloc[1]['Summarized Abstract']
    generated_summary = summaries[model_name]
    print_actual_vs_generated_summaries(model_name, actual_summary, generated_summary)


PEGASUS - Actual Summary:
This paper aims to evaluate whether there is a device-dependent effect on the                 reduction of surgical site complications in obese patients. The primary outcome of interest was surgical site infection (SSI) Secondary outcomes assessed included seroma, dehiscence, hematoma, bleeding, reoperation, readmission, and (composite) wound complications. A statistically significant improvement in the composite SSI and superficial SSI outcomes was observed with the -80 mm Hg device, compared with standard dressings.
PEGASUS - Generated Summary:
This study used 327 de-identified breast radiology reports from the anonymous institute.
The structured information was used to build the predictive model for predicting the value of the BIRADS category .


BASELINE - Actual Summary:
This paper aims to evaluate whether there is a device-dependent effect on the                 reduction of surgical site complications in obese patients. The primary outcome of interest w